In [1]:
%load_ext autoreload
%autoreload 2
import sys, os
from os.path import expanduser
## actions required!!!!!!!!!!!!!!!!!!!! change your folder path 
path_repo = expanduser("~/Documents/G3_2/regime-identification"); sys.path.append(path_repo)
path_file = expanduser("~/data/G3_2/regime-identification/simulation")
path = {}
for folder in ["summary", "best-idx", "latex", "summary-cluster", "best-idx-cluster", "latex-cluster"]:
    path[folder] = f"{path_file}/{folder}"

In [2]:
import pandas as pd
import numpy as np

In [3]:
from regime.simulation_helper import *

In [4]:
len_list = [[250, 500, 1000, 2000], [50, 100, 250, 500, 1000], [60, 120, 250, 500]]

# summary table & latex output

In [5]:
feat_dict = {'true': "HMM",
             'HMM-paper': "HMM",
             # 'HMM10init-paper': "HMM",
             # 'HMM1init': "HMM", 
             # 'HMM1initCovPrior': "HMM", 
             'discrete': "zhengB", 
             'cont': "zhengB",
             'cont-mode': "zhengB"}
name_dict = {'true': "true",
             "HMM-paper": "HMM",
             # 'HMM10init-paper': "$\\text{HMM}_{10}$", 
             # 'HMM1init': "HMM", 
             # 'HMM1initCovPrior': "HMM-prior", 
             'discrete': "discrete", 
             'cont': "cont",
             'cont-mode': "$\\text{cont}_{\\text{M}}$"}

In [6]:
save_results_in_latex(path, "3-state-daily", [500, 1000, 2000], feat_dict, name_dict, "paper", True)

'\\begin{table}[htbp]\n\n\\begin{adjustwidth}{-10cm}{-10cm} \n\\centering \n{\\fontsize{6}{8}\\selectfont\n\\begin{tabular}{l @{\\hspace{.2cm}} l @{\\hspace{.2cm}} l @{\\hspace{.2cm}} l @{\\hspace{.2cm}} l @{\\hspace{.2cm}} l @{\\hspace{.2cm}} l @{\\hspace{.2cm}} l @{\\hspace{.2cm}} l @{\\hspace{.2cm}} l}\n\\toprule\n & $\\mu_1$ & $\\mu_2$ & $\\mu_3$ & $\\sigma_1$ & $\\sigma_2$ & $\\sigma_3$ & $\\gamma_{12}$ & $\\gamma_{13}$ & $\\gamma_{21}$ \\\\\n\\midrule\n500 &  &  &  &  &  &  &  &  &  \\\\\ntrue & 0.0006(0.0000) & 0.0000(0.0000) & -0.0008(0.0000) & 0.0078(0.0000) & 0.0112(0.0000) & 0.0174(0.0000) & 0.0009(0.0000) & 0.0010(0.0000) & 0.0032(0.0000) \\\\\nHMM & 0.0009(0.0061) & 0.0012(0.0086) & -0.0017(0.0073) & \\textbf{0.0078}(0.0023) & 0.0082(0.0039) & 0.0100(0.0054) & 0.2601(0.3376) & 0.2120(0.3009) & 0.3909(0.3968) \\\\\ndiscrete & 0.0005(0.0015) & -0.0000(0.0029) & -0.0007(0.0031) & 0.0085(0.0027) & \\textbf{0.0115}(0.0033) & 0.0145(0.0042) & 0.0023(0.0093) & 0.0020(0.0069) & \\

In [73]:
n_s_dict = dict(zip(generate_key_data("t", dof=5), len_list))

In [74]:
for key_data, n_s_list in n_s_dict.items():
    _ = save_results_in_latex(path, key_data, n_s_list, feat_dict, name_dict, "all", True)

In [7]:
def save_results_in_latex(path, key_data, n_s_list, feat_dict, name_dict, latex_file_name, cluster=False):
    """
    only do it for one key_data.
    """
    latex_str = add_cluster("latex", cluster)
    df = generate_numerical_results_table(path, key_data, n_s_list, feat_dict, name_dict, cluster)
    string = print_table_to_latex(df, ".2cm", key_data)
    filename = generate_file_name(path, latex_str, key_data, latex_file_name, suffix="latex"); check_dir_exist(filename)
    with open(filename, "w") as f:
        f.write(string)
    return string

In [7]:
key_data = "3-state-daily"
n_s=250 
cluster=True

In [ ]:
def generate_numerical_results_table(path, key_data, n_s, feat_dict, name_dict, cluster=False):
    """
    n_s can be a list.
    """
    if isinstance(n_s, list):
        # return {n_s_: generate_numerical_results_table(path, key_data, n_s_, feat_dict, name_dict, cluster) for n_s_ in n_s}
        return pd.concat([generate_numerical_results_table(path, key_data, n_s_, feat_dict, name_dict, cluster) for n_s_ in n_s])
    # load best_idx
    best_idx_str, summary_str = add_cluster("best-idx", cluster), add_cluster("summary", cluster)
    idx_df = load_file(path, best_idx_str, key_data, "best_idx", suffix="csv", index_col=0)
    for model_name in name_dict:
        if "true" in model_name or "HMM" in model_name: idx_df[model_name] = 0
    means_combined_df = {new_name: load_file(path, summary_str, key_data, "means", n_s, feat_dict[model], model, suffix="h5").iloc[idx_df.loc[n_s, model]] \
                         for model, new_name in name_dict.items()}
    stds_combined_df = {new_name: load_file(path, summary_str, key_data, "stds", n_s, feat_dict[model], model, suffix="h5").iloc[idx_df.loc[n_s, model]] \
                         for model, new_name in name_dict.items()}
    means_combined_df, stds_combined_df = pd.DataFrame(means_combined_df).T, pd.DataFrame(stds_combined_df).T
    # accuracy of estimation
    df_acc_for_rank = {col: (means_combined_df[col]-means_combined_df.loc["true", col]).abs() if not col_is_acc(col) else -means_combined_df[col] for col in means_combined_df.columns}
    df_acc_for_rank = pd.DataFrame(df_acc_for_rank).drop("true")
    best_model_ser = df_acc_for_rank.idxmin()
    means_combined_df, stds_combined_df = means_combined_df.applymap(lambda x: f"{x:.4f}"), stds_combined_df.applymap(lambda x: f"{x:.4f}")
    for col in best_model_ser.index:
        means_combined_df.loc[best_model_ser[col], col] = "\\textbf{" + means_combined_df.loc[best_model_ser[col], col] + "}"
    # combine
    df_ret = combine_means_std_df(means_combined_df, stds_combined_df)
    df_ret = pd.concat([pd.DataFrame("", index=[n_s], columns=df_ret.columns), df_ret])
    return df_ret
#combine_summary_means_std_df(means_combined_df, stds_combined_df) #means_combined_df, stds_combined_df, df_acc_for_rank


In [60]:
def output_column_format(df_ret, space):
    n_col=df_ret.shape[1]
    return ("l @{\hspace{" + space + "}} ") * n_col + "l"

# in paper

In [19]:
feat_dict_s = {'true': "HMM",
             'HMM10init': "HMM", 
             'discrete': "zhengF", 
             'cont': "zhengF",
             'cont-mode': "zhengF"}
name_dict_s = {'true': "true",
             'HMM10init': "$\\text{HMM}_{10}$", 
             'discrete': "discrete", 
             'cont': "cont.",
             'cont-mode': "cont. mode"}

In [20]:
_ = save_results_in_latex(path, "2-state-daily", [250, 500, 1000], feat_dict_s, name_dict_s, "paper")

In [21]:
_ = save_results_in_latex(path, "2-state-weekly", [100, 250, 500], feat_dict_s, name_dict_s, "paper")

In [22]:
_ = save_results_in_latex(path, "2-state-monthly", [120, 250, 500], feat_dict_s, name_dict_s, "paper")

In [23]:
_ = save_results_in_latex(path, "t-daily", [250, 500, 1000, 2000], feat_dict_s, name_dict_s, "paper")